In [1]:
#enter file name
#needs event listener
file = 'westnile_dataset.csv'

In [2]:
#dependencies
import pandas as pd
import numpy as np

In [3]:
#drop unnecessary columns and rename keys
df = pd.read_csv(file)
df = df.loc[df['PartOfCumulativeCountSeries'] == 1]
df = df[['Admin1Name', 'PeriodStartDate', 'PeriodEndDate', 'CountValue']]
df = df.rename(columns={'Admin1Name':'State', 'PeriodStartDate':'Start', 'PeriodEndDate':'End', 'CountValue':'Count'})
df = df.loc[df['Start'] > '2009-12-31']
df.sort_values(by=['State', 'End', 'Start'], inplace=True)
df.reset_index(inplace=True, drop=True)

In [4]:
#create template dataframe with states
states = ['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 'DISTRICT OF COLUMBIA', 'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 'OREGON', 'PENNSYLVANIA', 'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING']
states_df = pd.DataFrame(states)
states_df.rename(columns={0:'State'}, inplace=True)
template_df = pd.DataFrame(states)
template_df.rename(columns={0:'State'}, inplace=True)

In [5]:
#assign 2010 census population to variable
USpop = 308745538

In [6]:
#determine final year of data collection and assign range to variable
finalyear = df['Start'].iloc[-1]
finalyear = int(finalyear[0:4])
timerange = np.arange(2010, finalyear+1)

In [7]:
#determine final cumulative count per year per state
for year in timerange:
    year_df = [[]]
    year_df = df.loc[(df.Start.str.startswith(str(year)))].append(pd.Series(index=()), ignore_index=True)
    year_data = []
    lastrow = len(year_df) - 1

    for index, column in year_df.iterrows():
        if (index < lastrow) and (year_df.iloc[index,0] != year_df.iloc[(index+1),0]):
            year_data.append(year_df.iloc[index])

    cleaned_year_df = pd.DataFrame()
    cleaned_year_df = pd.DataFrame(year_data)
    cleaned_year_df.reset_index(inplace=True, drop=True)
    cleaned_year_df.drop(columns=['Start', 'End'], inplace=True)
    cleaned_year_df = cleaned_year_df.rename(columns={'Count':'Cases_'+str(year)})

    #merge template dataframe with states & yearly cumulative data
    template_df = template_df.merge(cleaned_year_df, on='State', how='outer')
    merged_df = states_df.merge(template_df, on='State', how='left')

    data_df = merged_df

In [8]:
#fill in empty cells with 0
merged_df = merged_df.fillna(0)
merged_df.set_index('State', inplace=True, drop=True)

#add totals by state and by year
merged_df['TotalByState'] = merged_df.sum(axis=1)
merged_df.loc['TotalByYear'] = merged_df.sum()

#calculate rate and add to dataframe
for year in timerange:
    merged_df['Rate_'+str(year)] = merged_df['Cases_'+str(year)] / USpop * 100000

#export as csv file
merged_df.to_csv('cleaned_dataset.csv')

#create csv for line graph
line = pd.DataFrame(merged_df.loc['TotalByYear'])
line = line[line.index.str.contains('Cases') == True]
line['Year'] = timerange
line.set_index('Year', drop=True, inplace=True)
line.rename(columns={'TotalByYear':'Total'}, inplace=True)
line.to_csv('line_dataset.csv')

#create csv for data.html
data_df = data_df.fillna(0)
data_df.set_index('State', inplace=True, drop=True)
data_df.rename(columns = lambda x : str(x)[-4:], inplace=True)
data_df['TotalByState'] = data_df.sum(axis=1)
data_df.loc['TotalByYear'] = data_df.sum()
data_df.to_csv('data_dataset.csv')

In [41]:
#rates data for carto
cut = len(timerange) + 1
carto_df = merged_df.iloc[:,cut:]
carto_df.rename(columns = lambda x : str(x)[-4:], inplace=True)
carto_df = carto_df[:-1]

abbr = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "DC", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY",
        "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", 
        "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
carto_df['abbr'] = abbr
carto_df.set_index('abbr', inplace=True)
carto_df.to_csv('carto_dataset.csv')
carto_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,2010,2011,2012,2013,2014,2015,2016,2017
abbr,,,,,,,,
AL,0.000648,0.000000,0.002267,0.000000,0.000648,0.001619,0.002915,0.008421
AK,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AZ,0.018786,0.006478,0.003239,0.000648,0.007773,0.011336,0.006154,0.005830
AR,0.000324,0.000000,0.000648,0.000000,0.000648,0.000648,0.000972,0.000972
CA,0.012308,0.014575,0.043725,0.001943,0.081297,0.060244,0.028826,0.043078
